In [ ]:
# In a Kaggle code cell
!pip install anomalib
!pip install git+https://github.com/facebookresearch/detectron2.git # Required for the Mask/Pose R-CNN

In [ ]:
import os
import shutil

# 1. Define where you want the "Perfect" dataset to live
ROOT_DIR = '/kaggle/working/avenue_dataset_final'
TRAIN_DEST = os.path.join(ROOT_DIR, 'training_videos')
TEST_DEST = os.path.join(ROOT_DIR, 'testing_videos')

# 2. Create folders
os.makedirs(TRAIN_DEST, exist_ok=True)
os.makedirs(TEST_DEST, exist_ok=True)

# 3. Copy/Move your CLEAN/FIXED videos there
# Replace these paths with where your fixed videos currently are!
FIXED_TRAIN_SOURCE = '/kaggle/input/stae-cleaned/denoised_dataset_train' 
FIXED_TEST_SOURCE = '/kaggle/input/stae-cleaned/denoised_dataset_test'

print("Moving Training Videos...")
for vid in os.listdir(FIXED_TRAIN_SOURCE):
    src = os.path.join(FIXED_TRAIN_SOURCE, vid)
    dst = os.path.join(TRAIN_DEST, vid)
    if os.path.isdir(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)

print("Moving Testing Videos...")
for vid in os.listdir(FIXED_TEST_SOURCE):
    src = os.path.join(FIXED_TEST_SOURCE, vid)
    dst = os.path.join(TEST_DEST, vid)
    if os.path.isdir(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)

print(f"✅ Dataset ready at: {ROOT_DIR}")

In [ ]:
!pip install "numpy<2"

In [ ]:
import cv2
import os
import glob
import re
from tqdm import tqdm

# ================= CONFIGURATION =================
# 1. Input: Where your current frame folders are
SOURCE_ROOT = '/kaggle/working/avenue_dataset_final' 
# Structure expected inside SOURCE_ROOT:
#   /training_videos/01/frame_xxx.jpg
#   /testing_videos/01/frame_xxx.jpg

# 2. Output: Where the final .avi files will go
DEST_ROOT = '/kaggle/working/avenue_dataset_final2'

FPS = 25
# =================================================

def natural_sort_key(s):
    """Sorts strings with numbers naturally (e.g. frame_2 -> frame_10)"""
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', s)]

def convert_frames_to_video(source_dir, dest_dir):
    if not os.path.exists(source_dir):
        print(f"⚠️ Source not found: {source_dir}")
        return

    os.makedirs(dest_dir, exist_ok=True)
    video_folders = sorted([f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))])
    
    print(f"📂 Processing {len(video_folders)} videos from {source_dir}...")
    
    for vid_id in tqdm(video_folders):
        frames_path = os.path.join(source_dir, vid_id)
        
        # Get all images
        images = glob.glob(os.path.join(frames_path, '*.jpg')) + \
                 glob.glob(os.path.join(frames_path, '*.png'))
        
        # Sort properly
        images.sort(key=natural_sort_key)
        
        if not images:
            continue
            
        frame = cv2.imread(images[0])
        height, width, layers = frame.shape
        
        save_path = os.path.join(dest_dir, f"{vid_id}.avi")
        
        # Initialize Video Writer
        fourcc = cv2.VideoWriter_fourcc(*'MJPG') 
        video = cv2.VideoWriter(save_path, fourcc, FPS, (width, height))
        
        for img_path in images:
            video.write(cv2.imread(img_path))
        
        # --- FIXED HERE ---
        # Removed cv2.destroyAllWindows()
        video.release() 

if __name__ == "__main__":
    # 1. Convert Training Set
    print("--- Converting TRAINING Set ---")
    convert_frames_to_video(
        source_dir=os.path.join(SOURCE_ROOT, 'training_videos'),
        dest_dir=os.path.join(DEST_ROOT, 'training_videos')
    )
    
    # 2. Convert Testing Set
    print("\n--- Converting TESTING Set ---")
    convert_frames_to_video(
        source_dir=os.path.join(SOURCE_ROOT, 'testing_videos'),
        dest_dir=os.path.join(DEST_ROOT, 'testing_videos')
    )
    
    print(f"\n✅ Done! Dataset ready at: {DEST_ROOT}")

In [ ]:
!pip install av

In [2]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

# Import Anomalib components (New v1.0+ API)
from anomalib.engine import Engine
from anomalib.models import AiVad
from anomalib.data import Avenue

# ================= CONFIGURATION =================
DATASET_ROOT = "/kaggle/working/avenue_dataset_final2"
OUTPUT_CSV = "submission_aivad.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# =================================================

def run_aivad_pipeline():
    print("🚀 Initializing AI-VAD Pipeline...")

    # 1. Define DataModule
    # We point gt_dir to the same root to avoid path errors, 
    # even though we don't have masks.
    # Update your DataModule definition
    datamodule = Avenue(
        root="/kaggle/working/avenue_dataset_final2", # Your dataset path
        gt_dir=None,                                  # <--- SET THIS TO NONE
        image_size=(256, 448),
        train_batch_size=1,
        eval_batch_size=1,
        num_workers=2
    )
    
    # 2. Define Model (The "Super-Competitor" Config)
    model = AiVad(
        box_score_thresh=0.8,
        persons_only=True,               # <--- The key to ignoring background noise
        min_bbox_area=100,
        enable_foreground_detections=True,
        use_pose_features=True,          # Uses Skeleton
        use_velocity_features=True,      # Uses Optical Flow
        use_deep_features=True,          # Uses CLIP
        n_neighbors_pose=1,
        n_neighbors_deep=1
    )

    # 3. Create Engine
    # Note: We do NOT pass 'task' here to avoid the TypeError
    engine = Engine(
        accelerator="gpu",
        devices=1,
        default_root_dir="/kaggle/working/aivad_logs", # Save logs here
    )

    # 4. FIT (Train)
    print("\n🧠 Fitting Density Estimators (Training)...")
    # This extracts features from train videos and builds the memory bank
    # It is fast because it doesn't backpropagate through the CNNs
    engine.fit(model=model, datamodule=datamodule)

    # 5. PREDICT (Inference)
    print("\n👀 Running Inference on Test Set...")
    # predict() returns a list of dictionaries containing scores and metadata
    predictions = engine.predict(model=model, datamodule=datamodule)

    # 6. Parse Results & Create CSV
    print("\n📝 Generating Submission CSV...")
    
    submission_rows = []
    
    # Iterate through the predictions batch by batch
    for batch_item in tqdm(predictions):
        # batch_item keys usually: ['image_path', 'pred_scores', 'pred_masks', ...]
        
        # Get Scores (Tensor) -> Convert to List
        # Scores might be one per frame in the batch
        scores = batch_item["pred_scores"].cpu().numpy()
        
        # Get Paths to identify Video/Frame
        # image_path might be a list of paths
        paths = batch_item["image_path"]
        
        # If it's a single item batch, wrap in list if needed (engine usually batches)
        if isinstance(paths, str): paths = [paths]
        if np.ndim(scores) == 0: scores = [scores]

        for filepath, score in zip(paths, scores):
            # Filepath is likely: .../testing_videos/01/000.jpg (extracted by Anomalib)
            # OR .../testing_videos/01.avi (if processing video directly)
            
            # We need to extract Video ID and Frame Number
            # Anomalib often extracts frames internally to /tmp/ or processes clip info
            
            filename = os.path.basename(filepath) # e.g. "01.avi" or "00001.jpg"
            parent_dir = os.path.basename(os.path.dirname(filepath)) # e.g. "testing_videos" or "01"
            
            try:
                # Scenario A: Path is .../01/00012.jpg
                vid_id = int(parent_dir) # Folder name is video ID
                frame_num = int(os.path.splitext(filename)[0]) # Filename is frame num
            except ValueError:
                # Scenario B: Path is .../01.avi (Video level prediction?)
                # AI-VAD usually works on clips. If image_path points to video, 
                # we might need deeper inspection. 
                # HOWEVER, Avenue datamodule usually yields FRAMES.
                # Let's assume Scenario A structure which is standard for Anomalib Avenue.
                print(f"Skipping ambiguous path: {filepath}")
                continue

            # Add to list
            row_id = f"{vid_id}_{frame_num}"
            submission_rows.append({"ID": row_id, "Predicted": float(score)})

    # Create DataFrame
    df = pd.DataFrame(submission_rows)
    
    # 7. Normalize Scores (0 to 1)
    # AI-VAD produces density scores which can be large. We must normalize.
    if not df.empty:
        df['Predicted'] = (df['Predicted'] - df['Predicted'].min()) / (df['Predicted'].max() - df['Predicted'].min())
        
        # Sort nicely
        # Helper to sort by video then frame
        df[['vid', 'frm']] = df['ID'].str.split('_', expand=True).astype(int)
        df = df.sort_values(['vid', 'frm']).drop(columns=['vid', 'frm'])
        
        # Save
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"\n✅ Success! Saved {len(df)} rows to {OUTPUT_CSV}")
        print(df.head())
    else:
        print("❌ Error: No predictions generated. Check paths.")

if __name__ == "__main__":
    run_aivad_pipeline()

🚀 Initializing AI-VAD Pipeline...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores



🧠 Fitting Density Estimators (Training)...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
100%|██████████| 2/2 [00:08<00:00,  4.24s/it]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type          ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor  │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator     │      0 │ train │     0 │
│ 3 │ model          │ AiVadModel    │  260 M │ train │     0 │
└───┴────────────────┴───────────────┴────────┴───────┴───────┘

Trainable params: 259 M                                                                                            
Non-trainable params: 447 K                                                                                        
Total params: 260 M                                                                                                
Total estimated model params size (MB): 1.0 K                                                                      
Modules in train mode: 670                                                                                         
Modules in eval mode: 227                                                                                          
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py:534: Found 227 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/scipy/io/matlab/_mio.py", line 39, in _open_file
    return open(file_like, mode), True
           ^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/avenue_dataset_final2/testing_label_mask/1_label.mat'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anomalib/data/datasets/ba